Import libraries

In [2]:
!pip install beautifulsoup4
!pip install Selenium
!pip install webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [3]:
!apt-get update
!apt-get install chromium-driver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [908 kB]
Ge

In [4]:
import selenium
import requests
import time
import pandas as pd
import copy

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, JavascriptException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from pprint import pprint

#Mount the google drive
from google.colab import drive
import os

drive.mount("/content/gdrive")
path = "/content/gdrive/MyDrive/Colab Notebooks/csv"

os.chdir(path)

Mounted at /content/gdrive


# Scope of webscraping:


In [ ]:
def web_driver():
  options = webdriver.ChromeOptions()
  options.add_argument("--verbose")
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-gpu')
  options.add_argument("--window-size=1920, 1200")
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=options)
  return driver

driver = web_driver()

driver.implicitly_wait(30)

card_listings = []

parent_url = ""

while parent_url != "https://www.zigwheels.ph/used-cars?p=29500000-40000000" or button4 != None:

  if parent_url == "":
    parent_url = "https://www.zigwheels.ph/used-cars?p=5000001-40000000"

  driver.get(parent_url)

  action = ActionChains(driver)

  WebDriverWait(driver, 30).until(EC.invisibility_of_element_located((By.ID, "used_lead_overlay_loader")))

  button1 = driver.find_element(By.XPATH, "//button[@class='btn-line btn-mini btn-sort grey sort-button uc__dropdown desc']")
  button2 = driver.find_element(By.XPATH, "//input[@id='p-l-h']")
  #button3 = driver.find_element(By.XPATH, "//a[@id='btn-view-all']")

  button1.click()
  button2.click()
  #button3.click()

  # Scroll to the button1 element to make it visible
  driver.execute_script("arguments[0].scrollIntoView();", button1)
  action.move_to_element(to_element=button1)
  action.click(on_element=button1)

  # Wait for the element to become visible after clicking button1
  time.sleep(2) # Adjust the wait time as needed

  # Scroll to button2 and click
  driver.execute_script("arguments[0].scrollIntoView();", button2)
  action.move_to_element(to_element=button2)
  action.click(on_element=button2)

  #driver.execute_script("arguments[0].scrollIntoView();", button3)
  #driver.execute_script("arguments[0].click();", button3)

  time.sleep(2)

  try:
    action.perform()
  except JavascriptException:
    break

  soup = BeautifulSoup(driver.page_source, 'html.parser')

  car_cards = soup.find_all("li", class_="card splide__slide shadow-light filter-listing-card used-car-card")

  for car_card in car_cards:

    car_card_url = "https://www.zigwheels.ph%s" % (car_card.find("a")["href"])
    print(car_card_url)
    response = requests.get(car_card_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        page_content = response.text
    else:
        print("Failed to fetch the page. Status code:", response.status_code)

    soup = BeautifulSoup(page_content, "html.parser")

    # Columns for the DataFrame

    ul_class = soup.find("ul", class_="d-flex flex-wrap justify-space-between uc-detail-lists m-sm-b")

    entries = ul_class.find_all("li", class_ = "d-flex align-center m-md-b")

    # Extract the keys for the dictionary
    keys = []
    for entry in entries:
        img_tag = entry.find("img")
        if img_tag and 'alt' in img_tag.attrs:
            keys.append(img_tag['alt'].lower().replace(" ", "_"))

    # Extract the values of the "alt" attribute for the dictionary
    ul_class = soup.find("ul", class_="d-flex flex-wrap justify-space-between uc-detail-lists m-sm-b").find_all("span")

    # Create a dictionary from the keys and values

    car_card = {}

    model = soup.find("h2", class_="m-md-b").text
    price = soup.find("span", class_="f-24 f-bold vh-price").text
    car_card["model"] = model
    car_card["price"] = price

    for key in keys:
      car_card[key] = ul_class[keys.index(key)].text

    card_listings.append(car_card)
    time.sleep(1)

  url_split = parent_url.split("=")
  range_split = url_split[1].split("-")

  if "Million" in price:
    latest_price = price
    latest_price = latest_price.split(" ")[0]
    latest_price = latest_price.split("₱")[1]
    latest_price = int(float(latest_price) * 1000000)

    parent_url = url_split[0] + "=" + f"{latest_price}" + "-" + range_split[1]

  else:
    latest_price = price
    latest_price = latest_price.split(" ")[0]
    latest_price = latest_price.split("₱")[1]
    remove_comma = latest_price.split(",")
    latest_price = ""

    for str in remove_comma:
      latest_price += str

    parent_url = url_split[0] + "=" + latest_price + "-" + range_split[1]

  url_split = ""
  range_split = ""
  latest_price = ""

  try:
    button4 = driver.find_element(By.XPATH, "//a[@class='btn-view-all btn-line btn-more arrow-down']")
  except NoSuchElementException:
    button4 = None


# -----------------------------------------------------------------------------
# Create a DataFrame from the dictionary
print("Done!")
df = pd.DataFrame(card_listings)



https://www.zigwheels.ph/detail-used-cars/chevrolet-camaro-6-2l-at-38801
https://www.zigwheels.ph/detail-used-cars/shelby-mustang-5-0l-at-33429
https://www.zigwheels.ph/detail-used-cars/shelby-mustang-gt350-27568
https://www.zigwheels.ph/detail-used-cars/bmw-x5-xdrive30d-43565
https://www.zigwheels.ph/detail-used-cars/chevrolet-bt-cruiser-5-3l-29475
https://www.zigwheels.ph/detail-used-cars/mercedes-benz-s-class-2011-2020-s500-4-7l-44496
https://www.zigwheels.ph/detail-used-cars/porsche-718-boxster-pdk-48464
https://www.zigwheels.ph/detail-used-cars/mercedes-benz-gl-class-350-bluetec-4-matic-42347
https://www.zigwheels.ph/detail-used-cars/land-rover-range-rover-velar-s-d200-with-satin-pack-48870
https://www.zigwheels.ph/detail-used-cars/cadillac-escalade-esv-platinum-38898
https://www.zigwheels.ph/detail-used-cars/cadillac-escalade-esv-platinum-46503
https://www.zigwheels.ph/detail-used-cars/porsche-cayman-pdk-38800
https://www.zigwheels.ph/detail-used-cars/audi-rs5-coupe-2-9l-tfsi-359

Under 100,000

In [ ]:
df_100k = pd.read_csv("df_100k.csv")
display(df_100k)
df_100k.to_csv("df_100k.csv")

,Unnamed: 0,model,price,registration_year,make_year,fuel_type,body_type,kms_driven,ownership,transmission,city,seats,engine_displacement
0,0,1990 Nissan Sentra 1.6L SGX MT Detail,"₱45,000",1990,1990,Gasoline,Sedan,"61,233 Km",First Owner,Manual,Quezon City,NaN,NaN
1,1,2021 Suzuki Celerio CVT 1.0L Detail,"₱60,000",2024,2021,Gasoline,Hatchback,"12,000 Km",First Owner,Cvt,Pasig,5 seat,998 CC
2,2,1994 Nissan Vanette 2.0L Vanette MT Detail,"₱70,000",2023,1994,Gasoline,Van,"84,710 Km",Second Owner,Manual,Quezon City,NaN,NaN
3,3,2000 Nissan Sentra 1.6L Super Saloon MT Detail,"₱70,000",2024,2000,Gasoline,Sedan,"150,000 Km",Second Owner,Manual,Mandaluyong City,NaN,NaN
4,4,1997 Mazda 3 Sedan 2.0L Premium Detail,"₱75,000",2019,1997,Gasoline,Sedan,"90,000 Km",Third Owner,Automatic,san-fernando,5 seat,1998 CC
5,5,2016 Tata Indigo 1.4L Diesel Detail,"₱75,000",2016,2016,Diesel,Sedan,"109,000 Km",First Owner,Manual,Cebu City,5 seat,1405 CC
6,6,1997 Mitsubishi Lancer 1.6L GLXI Limited AT De...,"₱75,000",2019,1997,Gasoline,Sedan,"200,000 Km",Third Owner,Automatic,Tanza,NaN,NaN
7,7,1993 Honda Civic 1.6L ESI MT Detail,"₱80,000",2023,1993,Gasoline,Sedan,"10,000 Km",Second Owner,Manual,Tarlac City,NaN,NaN
8,8,1994 Honda Civic S Turbo CVT Honda Sensing Detail,"₱90,000",2020,1994,Gasoline,Sedan,"50,000 Km",Second Owner,Cvt,san-mateo,5 seat,1498 CC
9,9,2010 Nissan Sentra 1.3L GX MT Detail,"₱100,000",2010,2010,Gasoline,Sedan,"300,000 Km",Second Owner,Manual,Bacoor,NaN,NaN


Under 150,000

In [ ]:
df_150k = pd.read_csv("df_150k.csv")
display(df_150k)
df_150k.to_csv("df_150k.csv")

,Unnamed: 0,model,price,registration_year,make_year,fuel_type,body_type,kms_driven,ownership,transmission,city,seats,engine_displacement
0,0,1993 Toyota Corona 2.0L EX Saloon MT Detail,"₱115,000",1993.0,1993,Gasoline,Sedan,"120,000 Km",Second Owner,Manual,Cavite City,NaN,NaN
1,1,2005 Ford Focus Sedan 1.8L Ghia AT Detail,"₱120,000",2018.0,2005,Gasoline,Sedan,"100,000 Km",Third Owner,Automatic,Cainta,NaN,NaN
2,2,1997 Toyota Corolla 1.6L GLI AT Detail,"₱130,000",1997.0,1997,Gasoline,Sedan,"80,000 Km",Second Owner,Automatic,Bacoor,NaN,NaN
3,3,2000 Honda Civic 1.8 S CVT Detail,"₱135,000",2000.0,2000,Gasoline,Sedan,"120,000 Km",Third Owner,Cvt,Alaminos,5 seat,1799 CC
4,4,1996 Mazda E2000 2.0 MT Detail,"₱140,000",1996.0,1996,Gasoline,Van,"120,000 Km",First Owner,Manual,Lipa City,NaN,NaN
5,5,2020 Hyundai Reina GL 4AT Detail,"₱146,077",NaN,2020,Gasoline,Sedan,"30,000 Km",First Owner,Automatic,Antipolo,5 seat,1368 CC
6,6,2006 Honda City Hatchback RS CVT Detail,"₱150,000",2023.0,2006,Gasoline,Hatchback,150 Km,Second Owner,Cvt,Butuan,5 seat,1498 CC
7,7,1994 Nissan Bluebird 2.0 MT Detail,"₱150,000",2024.0,1994,Gasoline,Sedan,"100,000 Km",Second Owner,Manual,Antipolo,NaN,NaN


Under 200,000

In [ ]:
df_200k = pd.read_csv("df_200k.csv")
df_200k = df.drop_duplicates().reset_index(drop=True)
display(df_200k)
df_200k.to_csv("df_200k.csv")

,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,2019 Toyota Rush 1.5 G GR-S A/T Detail,"₱810,000",2019,7 seat,Gasoline,Suv,"90,000 Km",First Owner,Automatic,Las Piñas,1496 CC,NaN
1,2015 Isuzu mu-X 3.0 4x2 LS-A AT Detail,"₱810,000",2015,7 seat,Diesel,Suv,"60,000 Km",First Owner,Automatic,Las Piñas,2999 CC,NaN
2,2019 Toyota Rush 1.5 E AT Detail,"₱815,000",2019,7 seat,Gasoline,Suv,"35,000 Km",First Owner,Automatic,Paranaque,1496 CC,2023
3,2016 Isuzu mu-X 3.0 LS-A 4x2 AT Detail,"₱818,000",2016,7 seat,Diesel,Suv,"51,000 Km",First Owner,Automatic,Lapu-Lapu City,2999 CC,2016
4,2016 Toyota Innova 2.5L G AT Detail,"₱820,000",2016,8 seat,Diesel,Mpv,"60,000 Km",First Owner,Automatic,Pasig,2494 CC,2016
...,...,...,...,...,...,...,...,...,...,...,...,...
63,2019 Foton Transvan HR 15 Seater Detail,"₱850,000",2019,15 seat,Diesel,Van,"55,000 Km",Third Owner,Manual,Cainta,2771 CC,2023
64,2020 Toyota Rush 1.5 G GR-S A/T Detail,"₱850,000",2020,7 seat,Gasoline,Suv,"55,000 Km",First Owner,Automatic,Cavite City,1496 CC,2023
65,2020 Isuzu Traviz L Detail,"₱850,000",2020,3 seat,Diesel,Van,"3,200 Km",First Owner,Manual,Valenzuela,2499 CC,2024
66,2016 Ford Everest 2.0L Turbo Titanium Plus 4x2...,"₱850,000",2016,7 seat,Diesel,Suv,"45,000 Km",First Owner,Automatic,Pasig,1996 CC,2016


Under 250,000

In [ ]:
df_250k = pd.read_csv("df_250k.csv")
df_250k = df_250k.drop_duplicates().reset_index(drop=True)
display(df_250k)
df_250k.to_csv("df_250k.csv")

,Unnamed: 0,model,price,make_year,fuel_type,body_type,ownership,transmission,city,registration_year,seats,kms_driven,engine_displacement
0,0,2009 Hyundai Accent 1.5 GL CRDi MT Detail,"₱210,000",2009,Diesel,Sedan,First Owner,Manual,Mandaluyong City,NaN,NaN,NaN,NaN
1,1,2001 Honda CR-V RS eHEV Detail,"₱210,000",2001,Hybrid,Suv,Second Owner,E-cvt,Davao City,2014.0,5 seat,"120,000 Km",1993 CC
2,2,2014 Mitsubishi Mirage 1.2 MT Detail,"₱210,000",2014,Gasoline,Hatchback,First Owner,Manual,Antipolo,2023.0,5 seat,"60,000 Km",1193 CC
3,3,2003 Toyota Camry 2.4 V AT Detail,"₱215,000",2003,Gasoline,Sedan,First Owner,Automatic,Quezon City,2004.0,NaN,"35,000 Km",NaN
4,4,2011 Hyundai Getz 1.1 GL MT Detail,"₱220,000",2011,Gasoline,Hatchback,First Owner,Manual,Marikina City,NaN,NaN,"54,000 Km",NaN
5,5,2008 Honda City 1.3L S MT Detail,"₱220,000",2008,Gasoline,Sedan,Second Owner,Manual,Manila,2008.0,NaN,"90,000 Km",1298 CC
6,6,2019 Suzuki Carry Cab and Chasis 1.5L Detail,"₱220,000",2019,Gasoline,Pickup-trucks,First Owner,Manual,Manila,2023.0,2 seat,"30,000 Km",1462 CC
7,7,2003 Honda CR-V 2.4L AT Detail,"₱220,000",2003,Gasoline,Suv,First Owner,Automatic,Cebu City,2003.0,NaN,"100,000 Km",NaN
8,8,2008 Toyota Vios 1.5 G CVT Detail,"₱220,000",2008,Gasoline,Sedan,Second Owner,Cvt,Antipolo,2008.0,5 seat,"115,000 Km",1496 CC
9,9,2012 Ford Fiesta Sedan 1.5L Trend AT Detail,"₱220,000",2012,Gasoline,Sedan,Second Owner,Automatic,Quezon City,2012.0,5 seat,"102,000 Km",1498 CC


Under 300,000

In [ ]:
df_300k = pd.read_csv("df_300k.csv")
df_300k = df_300k.drop_duplicates().reset_index(drop=True)
display(df_300k)
df_300k.to_csv("df_300k.csv")

,Unnamed: 0,model,price,make_year,fuel_type,body_type,kms_driven,ownership,transmission,city,registration_year,seats,engine_displacement
0,0,2014 Toyota Hiace 2.5L Commuter MT Detail,"₱252,240",2014,Diesel,Van,"78,000 Km",First Owner,Manual,Antipolo,NaN,NaN,NaN
1,1,2004 Honda Civic V Turbo CVT Honda Sensing Detail,"₱255,000",2004,Gasoline,Sedan,"120,000 Km",Second Owner,Cvt,Quezon City,2022.0,5 seat,1498 CC
2,2,2004 Ford Escape 2.0L XLS AT Detail,"₱258,000",2004,Gasoline,Suv,"68,000 Km",First Owner,Automatic,Quezon City,2004.0,NaN,NaN
3,3,2016 Hyundai Eon 0.8 GLX 5 M/T Detail,"₱259,000",2016,Gasoline,Hatchback,"50,000 Km",First Owner,Manual,Nasugbu,2016.0,5 seat,814 CC
4,4,2010 Mitsubishi Galant 2.4L SE AT Detail,"₱260,000",2010,Gasoline,Sedan,"50,000 Km",Second Owner,Automatic,Dagupan,2010.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,77,2012 Toyota Vios 1.3 E CVT Detail,"₱300,000",2012,Gasoline,Sedan,"50,000 Km",Second Owner,Cvt,san-fernando,2012.0,5 seat,1329 CC
78,78,2013 Ford Focus Sedan 1.6L Ambiente MT Detail,"₱300,000",2013,Gasoline,Sedan,"65,000 Km",Second Owner,Manual,Paranaque,2013.0,NaN,NaN
79,79,2013 Hyundai Elantra 1.6 GL 6M/T Detail,"₱300,000",2013,Gasoline,Sedan,"120,000 Km",Third Owner,Manual,Las Piñas,2013.0,5 seat,1597 CC
80,80,2019 Suzuki Celerio CVT 1.0L Detail,"₱300,000",2019,Gasoline,Hatchback,"50,000 Km",First Owner,Cvt,Cagayan de Oro,2019.0,5 seat,998 CC


Under 350,000*

In [ ]:
df_350k = pd.read_csv("df_350k.csv")
df_350k = df_350k.drop_duplicates().reset_index(drop=True)
display(df_350k)
df_350k.to_csv("df_350k.csv")

,Unnamed: 0,model,price,registration_year,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement
0,0,2020 Hyundai Reina GL 5MT Detail,"₱304,000",2020.0,2020,5 seat,Gasoline,Sedan,"71,000 Km",First Owner,Manual,Cebu City,1368 CC
1,1,2016 Chevrolet Sail 1.3 LT MT Detail,"₱305,000",2016.0,2016,5 seat,Gasoline,Sedan,"71,000 Km",First Owner,Manual,Quezon City,1349 CC
2,2,2013 Nissan Almera 1.5 N-Sport AT Detail,"₱305,000",2013.0,2013,5 seat,Gasoline,Sedan,"100,000 Km",First Owner,Automatic,"Talisay City, Cebu",1498 CC
3,3,2012 Ford Fiesta Sedan 1.5L Trend AT Detail,"₱310,000",2012.0,2012,5 seat,Gasoline,Sedan,"50,000 Km",First Owner,Automatic,Davao City,1498 CC
4,4,2010 Honda Accord 2.4 S AT Detail,"₱310,000",2010.0,2010,5 seat,Gasoline,Sedan,"75,000 Km",Third Owner,Automatic,Taguig,2356 CC
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,74,2012 Ford Fiesta Hatchback 1.6L Sport AT Detail,"₱350,000",2023.0,2012,NaN,Gasoline,Hatchback,"35,000 Km",First Owner,Automatic,Batangas,NaN
75,75,2011 Isuzu Crosswind XL MT Detail,"₱350,000",2022.0,2011,5 seat,Diesel,Mpv,"200,000 Km",Second Owner,Manual,Davao,2499 CC
76,76,2007 Nissan Frontier 2.7L Bravado MT Detail,"₱350,000",NaN,2007,NaN,Diesel,Pickup-trucks,NaN,First Owner,Manual,Paranaque,NaN
77,77,2017 Toyota Vios 1.3 J MT Detail,"₱350,000",NaN,2017,5 seat,Gasoline,Sedan,"80,000 Km",First Owner,Manual,Quezon City,1329 CC


Under 400,000*

In [ ]:
df_400k = pd.read_csv("df_400k.csv")
df_400k = df_400k.drop_duplicates().reset_index(drop=True)
display(df_400k)
df_400k.to_csv("df_400k.csv")

,Unnamed: 0,model,price,registration_year,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement
0,0,2011 Kia Carens LX AT Detail,"₱355,000",2011.0,2011,7 seat,Diesel,Hatchback,1 Km,First Owner,Automatic,Lapu-Lapu City,1685 CC
1,1,2010 Honda Civic 1.8L V AT Detail,"₱355,000",2010.0,2010,NaN,Gasoline,Sedan,"70,000 Km",First Owner,Automatic,Manila,NaN
2,2,2016 Toyota Vios 1.3 E MT Detail,"₱355,000",2022.0,2016,5 seat,Gasoline,Sedan,"59,000 Km",First Owner,Manual,San Jose del Monte,1329 CC
3,3,2014 Honda City 1.5 V CVT Honda Sensing Detail,"₱355,000",2023.0,2014,5 seat,Gasoline,Sedan,"90,000 Km",Second Owner,Cvt,Manila,1498 CC
4,4,2012 Mitsubishi Adventure 2.5 GLX MT Detail,"₱358,000",2023.0,2012,7 seat,Diesel,Mpv,"3,000 Km",First Owner,Manual,Quezon City,2477 CC
5,5,2009 Ford Everest 2.5L XLT AT Detail,"₱358,000",NaN,2009,NaN,Diesel,Suv,"60,000 Km",First Owner,Automatic,Mandaluyong City,NaN
6,6,2014 Ford Focus Hatchback 2.0L Sport Gasoline ...,"₱358,000",2015.0,2014,NaN,Gasoline,Hatchback,"40,000 Km",First Owner,Automatic,Bacoor,1998 CC
7,7,2004 Suzuki Jimny JLX 1.3L-A/T Detail,"₱360,000",2004.0,2004,NaN,Gasoline,Suv,"80,000 Km",First Owner,NaN,Quezon City,NaN
8,8,2017 Mitsubishi Mirage 1.2 MT Detail,"₱360,000",2017.0,2017,5 seat,Gasoline,Hatchback,"52,000 Km",Second Owner,Manual,Quezon City,1193 CC
9,9,2016 Mitsubishi Mirage G4 GLX CVT Detail,"₱360,000",2017.0,2016,5 seat,Gasoline,Sedan,"40,000 Km",Second Owner,Cvt,Manila,1193 CC


Under 450,000*

In [ ]:
df_450k = pd.read_csv("df_450k.csv")
df_450k = df_450k.drop_duplicates().reset_index(drop=True)
display(df_450k)
df_450k.to_csv("df_450k.csv")

,Unnamed: 0,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,0,2018 Mitsubishi Mirage G4 GLS 1.2 CVT Detail,"₱409,000",2018,5 seat,Gasoline,Sedan,"36,000 Km",First Owner,Cvt,Quezon City,1193 CC,NaN
1,1,2019 Hyundai Accent 1.4 GL 6MT w/o Airbags Detail,"₱410,000",2019,5 seat,Gasoline,Sedan,"15,000 Km",Second Owner,Manual,Vigan,1368 CC,2019.0
2,2,2013 Kia Sportage 2.0 AT Diesel FWD Detail,"₱410,000",2013,5 seat,Diesel,Suv,"106,000 Km",First Owner,Automatic,Makati City,1995 CC,2013.0
3,3,2011 Toyota Innova 2.0L J MT Detail,"₱410,000",2011,NaN,Gasoline,Mpv,1 Km,First Owner,Manual,"Talisay City, Cebu",1998 CC,2011.0
4,4,2019 Suzuki APV GA 1.6L MT Detail,"₱410,000",2019,8 seat,Gasoline,Minivans,"70,000 Km",First Owner,Manual,Bacoor,1590 CC,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,100,2014 Mitsubishi Adventure 2.5 GLX MT Detail,"₱450,000",2014,7 seat,Diesel,Mpv,"130,000 Km",First Owner,Manual,Milaor,2477 CC,NaN
101,101,2013 Honda City 1.5L E AT Detail,"₱450,000",2013,NaN,Gasoline,Sedan,80 Km,Second Owner,Automatic,Mataas na Kahoy,1499 CC,2013.0
102,102,2005 Nissan Frontier 2.7L Titanium MT Detail,"₱450,000",2005,NaN,Diesel,Pickup-trucks,NaN,First Owner,Manual,Quezon City,NaN,NaN
103,103,2017 Toyota Wigo 1.0 G AT Detail,"₱450,000",2017,5 seat,Gasoline,Hatchback,NaN,First Owner,Automatic,Quezon City,998 CC,NaN


Under 500,000*

In [ ]:
df_500k = pd.read_csv("df_500k.csv")
df_500k = df_500k.drop_duplicates().reset_index(drop=True)
display(df_500k)
df_500k.to_csv("df_500k.csv")

,Unnamed: 0,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,0,2010 Subaru Forester 2.0 AT Detail,"₱455,000",2010,5 seat,Gasoline,Suv,1 Km,First Owner,Automatic,Pasig,1995 CC,NaN
1,1,2017 Suzuki Swift GL 1.2L CVT Detail,"₱455,000",2017,5 seat,Gasoline,Hatchback,"39,000 Km",First Owner,Cvt,"Talisay City, Cebu",1197 CC,2017.0
2,2,2012 Mitsubishi Adventure 2.5 GLS Sport MT Detail,"₱455,000",2012,NaN,Diesel,Mpv,NaN,First Owner,Manual,Cavite City,2494 CC,NaN
3,3,2016 Toyota Vios 1.3L E AT Detail,"₱455,000",2016,NaN,Gasoline,Sedan,"72,866 Km",First Owner,Automatic,Bacoor,1293 CC,NaN
4,4,2015 Chevrolet Spin 1.5L AT LTZ(Gas) Detail,"₱458,000",2015,7 seat,Gasoline,Mpv,"70,000 Km",First Owner,Automatic,Mandaluyong City,1485 CC,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,64,2015 Toyota Yaris 1.3L E AT Detail,"₱480,000",2015,NaN,Gasoline,Hatchback,NaN,First Owner,Automatic,Quezon City,NaN,NaN
65,65,2007 BMW 1 Series (Five Door) 118i Detail,"₱480,000",2007,4 seat,Gasoline,Hatchback,"50,000 Km",First Owner,Manual,Cebu City,1598 CC,2008.0
66,66,2016 Mitsubishi L300 2.5L FB Exceed MT Detail,"₱480,000",2016,NaN,Diesel,Mpv,"120,000 Km",Second Owner,Manual,Marilao,2488 CC,2022.0
67,67,2009 BMW 1 Series (Five Door) 118i Detail,"₱480,000",2009,4 seat,Gasoline,Hatchback,"80,000 Km",Second Owner,Manual,Quezon City,1598 CC,2009.0


Under 550,000*

In [ ]:
df_550k = pd.read_csv("df_550k.csv")
df_550k = df_550k.drop_duplicates().reset_index(drop=True)
display(df_550k)
df_550k.to_csv("df_550k.csv")

,Unnamed: 0,model,price,registration_year,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement
0,0,2013 Mazda CX-5 2.0L FWD Pro Detail,"₱508,000",2013.0,2013,5 seat,Gasoline,Crossover,"74,000 Km",First Owner,Automatic,Mandaluyong City,1998 CC
1,1,2017 Ford Ecosport 1.5 L Ambiente MT Detail,"₱508,000",2017.0,2017,5 seat,Gasoline,Crossover,"40,000 Km",First Owner,Manual,Bacolod City,1498 CC
2,2,2016 Mitsubishi Adventure 2.5 GLX MT Detail,"₱508,000",NaN,2016,7 seat,Diesel,Mpv,"54,000 Km",First Owner,Manual,Cainta,2477 CC
3,3,2020 Nissan Almera 1.5 E AT Detail,"₱508,000",NaN,2020,5 seat,Gasoline,Sedan,"28,000 Km",First Owner,Automatic,Cainta,1498 CC
4,4,2021 Toyota Wigo 1.0L G AT Detail,"₱508,000",2021.0,2021,NaN,Gasoline,Hatchback,"21,000 Km",First Owner,Automatic,San Pascual,995 CC
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,112,2003 Toyota Hiace Commuter Deluxe Detail,"₱550,000",2003.0,2003,15 seat,Diesel,Van,"55,000 Km",First Owner,Manual,Quezon City,2755 CC
113,113,2016 Mazda 3 Sedan 1.5 V Detail,"₱550,000",NaN,2016,NaN,Gasoline,Sedan,"50,000 Km",First Owner,NaN,Las Piñas,NaN
114,114,2019 Suzuki Swift 1.2L-A/T Detail,"₱550,000",NaN,2019,5 seat,Gasoline,Hatchback,NaN,First Owner,Automatic,Caloocan City,1197 CC
115,115,2017 Ford Ecosport 1.5 L Trend AT Detail,"₱550,000",NaN,2017,5 seat,Gasoline,Crossover,NaN,First Owner,Automatic,Makati City,1498 CC


Under 600,000*

In [ ]:
df_600k = pd.read_csv("df_600k.csv")
df_600k = df_600k.drop_duplicates().reset_index(drop=True)
display(df_600k)
df_600k.to_csv("df_600k.csv")

,Unnamed: 0,model,price,registration_year,make_year,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,seats
0,0,2017 Hyundai Grand Starex 2.5 GL MT Detail,"₱555,000",2017.0,2017,Diesel,Van,"123,000 Km",First Owner,Manual,Quezon City,2498 CC,NaN
1,1,2005 Mitsubishi Pajero GLS 3.8 V6 Gas 4WD AT D...,"₱555,000",2023.0,2005,Gasoline,Suv,"118,000 Km",Third Owner,Automatic,Mexico,NaN,NaN
2,2,2019 Toyota Vios 1.3 XE CVT Detail,"₱555,000",2019.0,2019,Gasoline,Sedan,1 Km,First Owner,Cvt,Lapu-Lapu City,1329 CC,5 seat
3,3,2016 Hyundai Tucson 2.0L S MT Detail,"₱558,000",2016.0,2016,Gasoline,Suv,"50,000 Km",First Owner,Manual,Las Piñas,1998 CC,NaN
4,4,2020 Hyundai Accent 1.4 GL 6AT Detail,"₱558,000",NaN,2020,Gasoline,Sedan,1 Km,First Owner,Automatic,Quezon City,1399 CC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,120,2014 Toyota Innova 2.5L E AT Detail,"₱598,000",NaN,2014,Diesel,Mpv,NaN,First Owner,Automatic,Cavite City,2494 CC,NaN
121,121,2019 MG ZS Style AT Detail,"₱598,000",NaN,2019,Gasoline,Crossover,NaN,First Owner,Automatic,Paranaque,1499 CC,5 seat
122,122,2016 Isuzu Crosswind Sportivo MT Detail,"₱598,000",NaN,2016,Diesel,Mpv,NaN,First Owner,Manual,Cavite City,2499 CC,7 seat
123,123,2013 Chevrolet Colorado 2.8L 4X4 AT LTZ Detail,"₱598,000",NaN,2013,Diesel,Pickup-trucks,NaN,First Owner,Automatic,Cavite City,2766 CC,4 seat


Under 650,000*

In [ ]:
df_650k = pd.read_csv("df_650k.csv")
df_650k = df_650k.drop_duplicates().reset_index(drop=True)
display(df_650k)
df_650k.to_csv("df_650k.csv")

,Unnamed: 0,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,0,2019 Foton View Transvan 2.8L MT Detail,"₱605,000",2019,15 seat,Diesel,Van,"40,000 Km",First Owner,Manual,Quezon City,2800 CC,NaN
1,1,2018 Honda City 1.5 E CVT Detail,"₱608,000",2018,5 seat,Gasoline,Sedan,"21,000 Km",First Owner,Cvt,Quezon City,1496 CC,NaN
2,2,2017 Honda City 1.5 VX CVT Detail,"₱608,000",2017,5 seat,Gasoline,Sedan,"69,000 Km",First Owner,Automatic,Quezon City,1496 CC,NaN
3,3,2020 Toyota Vios 1.3L XLE AT Detail,"₱608,000",2020,NaN,Gasoline,Sedan,"13,000 Km",First Owner,Automatic,Pasig,1293 CC,NaN
4,4,2018 Hyundai H-100 2.6 GL 5M/T (Dsl-With AC) D...,"₱610,000",2018,NaN,Diesel,Pickup-trucks,"57,000 Km",First Owner,Manual,Quezon City,2607 CC,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,98,2018 Honda City 1.5L VX AT Detail,"₱650,000",2018,NaN,Gasoline,Sedan,"35,000 Km",First Owner,Automatic,Cebu City,NaN,2018.0
99,99,2017 Ford Ecosport 1.5L Trend AT Detail,"₱650,000",2017,NaN,Gasoline,Suv,"3,330 Km",First Owner,Automatic,Quezon City,1498 CC,2022.0
100,100,2020 Suzuki Swift GL 1.2L CVT Detail,"₱650,000",2020,5 seat,Gasoline,Hatchback,"11,800 Km",First Owner,Cvt,Quezon City,1197 CC,2020.0
101,101,2017 Hyundai Grand Starex 2.5 GL 5M/T (Dsl) De...,"₱650,000",2017,10 seat,Diesel,Minivans,"30,000 Km",First Owner,Manual,Manila,2476 CC,2022.0


Under 700,000

In [ ]:
df_700k = pd.read_csv("df_700k.csv")
df_700k = df_700k.drop_duplicates().reset_index(drop=True)
display(df_700k)
df_700k.to_csv("df_700k.csv")

,Unnamed: 0,model,price,registration_year,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement
0,0,2022 Mitsubishi Mirage G4 GLX CVT Detail,"₱655,000",2022.0,2022,5 seat,Gasoline,Sedan,"2,000 Km",First Owner,Cvt,Lapu-Lapu City,1193 CC
1,1,2022 Mitsubishi Mirage G4 GLX 1.2 CVT Detail,"₱655,000",2022.0,2022,NaN,Gasoline,Sedan,"1,000 Km",First Owner,Cvt,Quezon City,1199 CC
2,2,2014 Mitsubishi Strada 2.5 GLX MT Detail,"₱655,000",NaN,2014,NaN,Diesel,Pickup-trucks,"72,000 Km",First Owner,Manual,"Talisay City, Cebu",2498 CC
3,3,2020 MG ZS Alpha AT Detail,"₱655,000",2020.0,2020,5 seat,Gasoline,Crossover,"20,000 Km",First Owner,Automatic,Taguig,1499 CC
4,4,2017 Honda Mobilio 1.5 RS Navi CVT Detail,"₱655,000",2017.0,2017,7 seat,Gasoline,Mpv,"38,000 Km",First Owner,Cvt,Pulilan,1497 CC
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,126,2016 Chevrolet Trailblazer Premier Detail,"₱700,000",2017.0,2016,5 seat,Gasoline,Suv,"75,000 Km",Second Owner,Cvt,Cebu City,1341 CC
127,127,2011 Toyota Hiace 2.5L Commuter MT Detail,"₱700,000",2011.0,2011,NaN,Diesel,Van,"348,654 Km",First Owner,Manual,Tagbilaran City,NaN
128,128,2016 Hyundai Tucson 2.0 GLS Plus AT (Diesel) D...,"₱700,000",2023.0,2016,5 seat,Diesel,Suv,"55,000 Km",First Owner,Automatic,Taguig,1998 CC
129,129,2018 Honda Jazz 1.5 VX Navi CVT Detail,"₱700,000",2018.0,2018,5 seat,Gasoline,Hatchback,"36,000 Km",First Owner,Cvt,"Santa Rosa, Laguna",1497 CC


Under 750,000*

In [ ]:
df_750k = pd.read_csv("df_750k.csv")
df_750k = df_750k.drop_duplicates().reset_index(drop=True)
display(df_750k)
df_750k.to_csv("df_750k.csv")

,Unnamed: 0,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,0,2015 Hyundai Grand Starex GL TCI(10s)- U MT De...,"₱708,000",2015,10 seat,Gasoline,Minivans,1 Km,First Owner,Manual,Quezon City,2476 CC,NaN
1,1,2015 Honda HR-V 1.8 E CVT Detail,"₱710,000",2015,5 seat,Gasoline,Crossover,"12,000 Km",First Owner,Cvt,Manila,1799 CC,2017.0
2,2,2015 Honda HR-V 1.8L EL AT Detail,"₱710,000",2015,NaN,Gasoline,Suv,"70,000 Km",First Owner,Automatic,Las Piñas,NaN,NaN
3,3,2022 Toyota Vios 1.3 XLE CVT Detail,"₱710,000",2022,5 seat,Gasoline,Sedan,"16,000 Km",First Owner,Cvt,Puerto Princesa City,1329 CC,NaN
4,4,2019 Toyota Rush 1.5 G GR-S A/T Detail,"₱710,000",2019,7 seat,Gasoline,Suv,"35,000 Km",First Owner,Automatic,Quezon City,1496 CC,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,2015 Ford Ranger 2.0L Turbo XLT 4x2 AT Detail,"₱750,000",2015,5 seat,Diesel,Pickup-trucks,"95,000 Km",First Owner,Automatic,Marilao,1996 CC,2023.0
86,86,2023 Toyota Raize 1.2 E CVT Detail,"₱750,000",2023,5 seat,Gasoline,Crossover,"4,400 Km",First Owner,Cvt,Taguig,1198 CC,2023.0
87,87,2016 Chevrolet Trailblazer LT Detail,"₱750,000",2016,5 seat,Gasoline,Suv,"35,000 Km",First Owner,Cvt,Quezon City,1341 CC,2023.0
88,88,2018 Nissan NV350 Urvan Premium M/T 15-Seater ...,"₱750,000",2018,15 seat,Diesel,Minivans,"35,000 Km",First Owner,Manual,Bauan,2488 CC,2018.0


Under 800,000*

In [ ]:
df_800k = df.copy()
df_800k = df_800k.drop_duplicates().reset_index(drop=True)
display(df_800k)
df_800k.to_csv("df_800k.csv")

,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,2018 Nissan Juke 1.6 Upper CVT Detail,"₱758,000",2018,5 seat,Gasoline,Crossover,"5,000 Km",First Owner,Cvt,Pasig,1598 CC,NaN
1,2020 Hyundai Kona 2.0 GLS 6A/T Detail,"₱758,000",2020,5 seat,Gasoline,Crossover,1 Km,First Owner,Automatic,Pasig,1998 CC,NaN
2,2020 Hyundai Kona 2.0 GLS 6A/T Detail,"₱758,000",2020,5 seat,Gasoline,Crossover,"13,000 Km",First Owner,Automatic,Marikina City,1998 CC,2020
3,2020 Toyota Vios 1.5 G Prime CVT Detail,"₱758,000",2020,5 seat,Gasoline,Sedan,1 Km,First Owner,Cvt,Pasig,1496 CC,NaN
4,2019 Mazda 3 Sedan SkyActiv V Detail,"₱759,000",2019,5 seat,Gasoline,Sedan,"55,000 Km",Second Owner,Automatic,Makati City,1496 CC,2019
...,...,...,...,...,...,...,...,...,...,...,...,...
85,2020 Mitsubishi Xpander Cross 1.5L AT Detail,"₱800,000",2020,7 seat,Gasoline,Mpv,"30,000 Km",First Owner,Automatic,Paranaque,1499 CC,2020
86,2018 Mitsubishi Xpander GLS AT Detail,"₱800,000",2018,7 seat,Gasoline,Mpv,"46,000 Km",First Owner,Automatic,Imus,1499 CC,2018
87,2020 Peugeot 3008 1.6L Petrol Detail,"₱800,000",2020,5 seat,Gasoline,Crossover,"5,000 Km",First Owner,Automatic,Taguig,1598 CC,2020
88,2020 Mitsubishi Xpander GLX AT Detail,"₱800,000",2020,7 seat,Gasoline,Mpv,"80,000 Km",First Owner,Automatic,Bacoor,1499 CC,2020


Under 850,000*

In [ ]:
df_850k = df.copy()
df_850k = df_850k.drop_duplicates().reset_index(drop=True)
display(df_850k)
df_850k.to_csv("df_850k.csv")

,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,2019 Toyota Rush 1.5 G GR-S A/T Detail,"₱810,000",2019,7 seat,Gasoline,Suv,"90,000 Km",First Owner,Automatic,Las Piñas,1496 CC,NaN
1,2015 Isuzu mu-X 3.0 4x2 LS-A AT Detail,"₱810,000",2015,7 seat,Diesel,Suv,"60,000 Km",First Owner,Automatic,Las Piñas,2999 CC,NaN
2,2019 Toyota Rush 1.5 E AT Detail,"₱815,000",2019,7 seat,Gasoline,Suv,"35,000 Km",First Owner,Automatic,Paranaque,1496 CC,2023
3,2016 Isuzu mu-X 3.0 LS-A 4x2 AT Detail,"₱818,000",2016,7 seat,Diesel,Suv,"51,000 Km",First Owner,Automatic,Lapu-Lapu City,2999 CC,2016
4,2016 Toyota Innova 2.5L G AT Detail,"₱820,000",2016,8 seat,Diesel,Mpv,"60,000 Km",First Owner,Automatic,Pasig,2494 CC,2016
...,...,...,...,...,...,...,...,...,...,...,...,...
63,2019 Foton Transvan HR 15 Seater Detail,"₱850,000",2019,15 seat,Diesel,Van,"55,000 Km",Third Owner,Manual,Cainta,2771 CC,2023
64,2020 Toyota Rush 1.5 G GR-S A/T Detail,"₱850,000",2020,7 seat,Gasoline,Suv,"55,000 Km",First Owner,Automatic,Cavite City,1496 CC,2023
65,2020 Isuzu Traviz L Detail,"₱850,000",2020,3 seat,Diesel,Van,"3,200 Km",First Owner,Manual,Valenzuela,2499 CC,2024
66,2016 Ford Everest 2.0L Turbo Titanium Plus 4x2...,"₱850,000",2016,7 seat,Diesel,Suv,"45,000 Km",First Owner,Automatic,Pasig,1996 CC,2016


Under 900,000

In [ ]:
df_900k = df.copy()
df_900k = df_900k.drop_duplicates().reset_index(drop=True)
display(df_900k)
df_900k.to_csv("df_900k.csv")

,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,2019 Mitsubishi Xpander GLS Sport 1.5G 2WD AT ...,"₱857,000",2019,7 seat,Gasoline,Mpv,1 Km,First Owner,Automatic,Pasig,1499 CC,NaN
1,2021 Mitsubishi Xpander GLS Sport 1.5G 2WD AT ...,"₱858,000",2021,7 seat,Gasoline,Mpv,"5,000 Km",First Owner,Automatic,Marikina City,1499 CC,2021
2,2014 Mercedes-Benz B-Class 180 Detail,"₱858,000",2014,5 seat,Gasoline,Hatchback,"30,000 Km",First Owner,Automatic,Las Piñas,1595 CC,2014
3,2017 Chevrolet Trailblazer 2.8 2WD 6AT LT Detail,"₱858,000",2017,7 seat,Diesel,Suv,"50,000 Km",First Owner,Automatic,Quezon City,2776 CC,NaN
4,2019 Mitsubishi Xpander GLS 1.5G 2WD AT Detail,"₱858,000",2019,7 seat,Gasoline,Mpv,1 Km,First Owner,Automatic,Quezon City,1499 CC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
109,2019 Mitsubishi Xpander GLS Sport 1.5G 2WD AT ...,"₱900,000",2019,7 seat,Gasoline,Mpv,"11,000 Km",Second Owner,Automatic,Las Piñas,1499 CC,2019
110,2017 Chevrolet Trailblazer Premier Detail,"₱900,000",2017,5 seat,Gasoline,Suv,"40,000 Km",First Owner,Cvt,Mandaluyong City,1341 CC,2018
111,2007 Audi Q7 2.0 TFSI Quattro Detail,"₱900,000",2007,7 seat,Gasoline,Suv,"70,000 Km",Second Owner,Automatic,Pasig,1998 CC,2017
112,2022 Kia Seltos 1.5 LX Detail,"₱900,000",2022,5 seat,Gasoline,Suv,"17,000 Km",First Owner,Ivt,Makati City,1499 CC,2022


Under 950,000*

In [ ]:
df_950k = df.copy()
df_950k = df_950k.drop_duplicates().reset_index(drop=True)
display(df_950k)
df_950k.to_csv("df_950k.csv")

,model,price,registration_year,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement
0,2019 Mitsubishi Strada GLS 2WD MT Detail,"₱908,000",2019,2019,5 seat,Diesel,Pickup-trucks,"32,000 Km",First Owner,Manual,Lapu-Lapu City,2396 CC
1,2018 Toyota Hilux 2.4L G 2WD MT Detail,"₱910,000",2023,2018,5 seat,Diesel,Pickup-trucks,"93,000 Km",First Owner,Manual,Quezon City,2498 CC
2,2015 Toyota Fortuner Dsl AT 4x2 2.5 V Detail,"₱915,000",NaN,2015,7 seat,Diesel,Suv,"54,000 Km",First Owner,Automatic,Lumban,2494 CC
3,2022 Toyota Avanza 1.3 E MT Detail,"₱915,450",2022,2022,7 seat,Gasoline,Mpv,"28,000 Km",First Owner,Manual,Iloilo City,1329 CC
4,2020 Nissan NV350 Urvan Premium M/T 15-Seater ...,"₱916,000",2020,2020,15 seat,Diesel,Minivans,"222,000 Km",First Owner,Manual,Cebu City,2488 CC
...,...,...,...,...,...,...,...,...,...,...,...,...
68,2012 Ford Expedition 5.4L XLT AT Detail,"₱950,000",NaN,2012,NaN,Gasoline,Suv,1 Km,First Owner,Automatic,Quezon City,NaN
69,2019 Hyundai Kona 2.0 GLS 6A/T Detail,"₱950,000",2019,2019,5 seat,Gasoline,Crossover,"45,000 Km",First Owner,Automatic,Cebu City,1998 CC
70,2021 Toyota Hilux 2.4L G 2WD MT Detail,"₱950,000",2023,2021,5 seat,Diesel,Pickup-trucks,"25,000 Km",Second Owner,Manual,Manila,2498 CC
71,2018 Ford Everest 2.0L Turbo Trend 4x2 AT Detail,"₱950,000",2023,2018,7 seat,Diesel,Suv,"25,000 Km",First Owner,Automatic,Manila,1996 CC


Under 1,000,000

In [ ]:
df_1m = df.copy()
df_1m = df_1m.drop_duplicates().reset_index(drop=True)
display(df_1m)
df_1m.to_csv("df_1m.csv")

,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,2018 Toyota Hiace Commuter 3.0 M/T Detail,"₱955,000",2018,15 seat,Diesel,Van,"42,323 Km",First Owner,Manual,Marikina City,2982 CC,NaN
1,2020 Mitsubishi Xpander Cross AT Detail,"₱958,000",2020,7 seat,Gasoline,Mpv,"15,000 Km",First Owner,Automatic,Quezon City,1499 CC,NaN
2,2017 Chevrolet Trailblazer 2.8 2WD 6AT LT Detail,"₱958,000",2017,7 seat,Diesel,Suv,1 Km,First Owner,Automatic,Pasig,2776 CC,NaN
3,2020 Ford Ranger 2.2L XLS 4x2 AT Detail,"₱958,000",2020,5 seat,Diesel,Pickup-trucks,"18,000 Km",First Owner,Automatic,Pasig,2198 CC,2019
4,2021 Mitsubishi Xpander Cross 1.5L AT Detail,"₱958,000",2021,7 seat,Gasoline,Mpv,"35,000 Km",First Owner,Automatic,Paranaque,1499 CC,2023
...,...,...,...,...,...,...,...,...,...,...,...,...
80,2017 Mitsubishi Montero Sport GLS 2WD AT Detail,₱1 Million,2017,7 seat,Diesel,Suv,"23,500 Km",First Owner,Automatic,Quezon City,2442 CC,2023
81,2015 Toyota Fortuner 2.4 G Diesel 4x2 AT Detail,₱1 Million,2015,7 seat,Diesel,Suv,"15,000 Km",First Owner,Automatic,Cainta,2393 CC,2023
82,2022 Ford Territory Titanium Detail,₱1 Million,2022,5 seat,Gasoline,Suv,"20,000 Km",First Owner,Automatic,Makati City,1499 CC,2022
83,2021 Ford Ranger 2.0L Turbo Wildtrak 4x2 AT De...,₱1 Million,2021,5 seat,Diesel,Pickup-trucks,"12,200 Km",First Owner,Automatic,Paranaque,1996 CC,2021


Under 1,100,000*

In [ ]:
df_1_1m = df.copy()
df_1_1m = df_1_1m.drop_duplicates().reset_index(drop=True)
display(df_1_1m)
df_1_1m.to_csv("df_1_1m.csv")

,model,price,registration_year,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement
0,2018 Toyota Hilux 2.4L G 2WD AT Detail,₱1 Million,2018,2018,5 seat,Diesel,Pickup-trucks,"79,624 Km",First Owner,Automatic,Manila,2498 CC
1,2017 Mitsubishi Montero Sport 2.5 GLS V 4x2 AT...,₱1.02 Million,NaN,2017,7 seat,Diesel,Suv,"40,000 Km",First Owner,Automatic,Las Piñas,2477 CC
2,2017 Toyota Fortuner Gas AT 4x2 2.7 G Detail,₱1.02 Million,NaN,2017,7 seat,Gasoline,Suv,"50,000 Km",First Owner,Automatic,Las Piñas,2694 CC
3,2011 Toyota Hiace 2.5 L Super Grandia AT Detail,₱1.02 Million,NaN,2011,NaN,Diesel,Van,"96,000 Km",First Owner,Automatic,Pasig,2498 CC
4,2019 Toyota Hiace 3.0L Commuter MT Detail,₱1.02 Million,NaN,2019,15 seat,Diesel,Van,"50,000 Km",First Owner,Manual,Marilao,2982 CC
...,...,...,...,...,...,...,...,...,...,...,...,...
109,2019 Mazda CX-9 2.5L Turbo AWD Exclusive Detail,₱1.1 Million,2019,2019,7 seat,Gasoline,Suv,"95,000 Km",First Owner,Automatic,Mandaluyong City,2488 CC
110,2023 Hyundai Tucson 2.0L GLS AT Detail,₱1.1 Million,2023,2023,5 seat,Gasoline,Suv,"23,027 Km",First Owner,Automatic,Manila,1999 CC
111,2021 Toyota Innova 2.8L V AT Detail,₱1.1 Million,2021,2021,NaN,Diesel,Mpv,"35,000 Km",First Owner,Automatic,Makati City,NaN
112,2015 Hyundai Grand Starex 2.5 CRDi GLS 5AT Urb...,₱1.1 Million,2015,2015,9 seat,Diesel,Minivans,"100,000 Km",First Owner,Automatic,Naga,2497 CC


Under 1,200,000

In [ ]:
df_1_2m = df.copy()
df_1_2m = df_1_2m.drop_duplicates().reset_index(drop=True)
display(df_1_2m)
df_1_2m.to_csv("df_1_2m.csv")

,model,price,make_year,seats,fuel_type,body_type,ownership,transmission,city,engine_displacement,registration_year,kms_driven
0,2023 Toyota Veloz 1.5L G Detail,₱1.108 Million,2023,7 seat,Gasoline,Suv,First Owner,Cvt,Pasig,1496 CC,NaN,NaN
1,2017 Toyota Fortuner 2.4L G AT Detail,₱1.12 Million,2017,7 seat,Diesel,Suv,First Owner,Automatic,Quezon City,2393 CC,2017,"30,000 Km"
2,2022 Ford Territory Titanium Plus Detail,₱1.12 Million,2022,5 seat,Gasoline,Crossover,First Owner,Cvt,Las Piñas,1490 CC,NaN,"16,000 Km"
3,2020 Suzuki Jimny GL Monotone 5MT Detail,₱1.128 Million,2020,4 seat,Gasoline,Suv,First Owner,Manual,Pasig,1462 CC,NaN,1 Km
4,2019 Mitsubishi Montero Sport GLX 2WD MT Detail,₱1.128 Million,2019,7 seat,Diesel,Suv,First Owner,Manual,Quezon City,2442 CC,NaN,"41,000 Km"
...,...,...,...,...,...,...,...,...,...,...,...,...
64,2019 Subaru XV GT Edition Detail,₱1.2 Million,2019,5 seat,Gasoline,Crossover,First Owner,Cvt,Manila,1995 CC,2020,"39,000 Km"
65,2016 BMW 5 Series Gran Turismo 520d Detail,₱1.2 Million,2016,5 seat,Diesel,Sedan,First Owner,Automatic,Quezon City,1995 CC,2016,"82,000 Km"
66,2020 Suzuki Jimny GLX 2 Tone 4AT Detail,₱1.2 Million,2020,4 seat,Gasoline,Suv,First Owner,Automatic,Cebu City,1462 CC,2021,"28,000 Km"
67,2018 Ford Everest 2.0L Turbo Titanium Plus 4x2...,₱1.2 Million,2018,7 seat,Diesel,Suv,First Owner,Automatic,Dasmariñas,1996 CC,2023,"38,000 Km"


Under 1,300,000

In [ ]:
df_1_3m = df.copy()
df_1_3m = df_1_3m.drop_duplicates().reset_index(drop=True)
display(df_1_3m)
df_1_3m.to_csv("df_1_3m.csv")

,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,2018 Mitsubishi Montero Sport GLS 2WD AT Detail,₱1.208 Million,2018,7 seat,Diesel,Suv,1 Km,First Owner,Automatic,Quezon City,2442 CC,NaN
1,2022 Ford Ranger XLT 2.2L 4x2 AT Detail,₱1.218 Million,2022,NaN,Diesel,Pickup-trucks,"2,000 Km",First Owner,Automatic,Mandaue City,NaN,NaN
2,2019 Nissan Terra 2.5 4x2 VE AT Detail,₱1.22 Million,2019,7 seat,Diesel,Suv,"30,000 Km",First Owner,Automatic,Las Piñas,2488 CC,NaN
3,2018 Toyota Fortuner 2.4L G AT Detail,₱1.22 Million,2018,7 seat,Diesel,Suv,"45,000 Km",First Owner,Automatic,Cainta,2393 CC,NaN
4,2018 Toyota Fortuner 2.4L G AT Detail,₱1.225 Million,2018,7 seat,Diesel,Suv,"37,000 Km",First Owner,Automatic,Paranaque,2393 CC,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
68,2019 Toyota Fortuner 2.4 G Diesel 4x2 AT Detail,₱1.298 Million,2019,7 seat,Diesel,Suv,"35,000 Km",First Owner,Automatic,Pasig,2393 CC,NaN
69,2017 Honda CR-V S-Diesel 9AT Honda Sensing Detail,₱1.3 Million,2017,7 seat,Diesel,Suv,"40,000 Km",First Owner,Automatic,San Mariano,1597 CC,2017
70,2015 Toyota Fortuner 2.4 V Diesel 4x2 AT Detail,₱1.3 Million,2015,7 seat,Diesel,Suv,"15,400 Km",First Owner,Automatic,Pasig,2393 CC,2015
71,2017 Toyota Fortuner 2.4L G AT Detail,₱1.3 Million,2017,7 seat,Diesel,Suv,"49,000 Km",First Owner,Automatic,Taal,2393 CC,NaN


Under 1,400,000

In [ ]:
df_1_4m = df.copy()
df_1_4m = df_1_4m.drop_duplicates().reset_index(drop=True)
display(df_1_4m)
df_1_4m.to_csv("df_1_4m.csv")

,model,price,make_year,fuel_type,body_type,kms_driven,ownership,transmission,city,seats,engine_displacement,registration_year
0,2017 Hyundai Grand Starex 2.5 CVX Detail,₱1.32 Million,2017,Diesel,Van,"21,000 Km",First Owner,Automatic,Valenzuela,NaN,NaN,NaN
1,2020 Toyota Fortuner 2.4 G Diesel 4x2 AT Detail,₱1.32 Million,2020,Diesel,Suv,"28,000 Km",First Owner,Automatic,Caloocan City,7 seat,2393 CC,NaN
2,2016 Toyota Fortuner 2.4 V Diesel 4x2 AT Detail,₱1.325 Million,2016,Diesel,Suv,"30,000 Km",First Owner,Automatic,Quezon City,7 seat,2393 CC,2023
3,2018 Ford Everest Titanium 2.2L 4x2 AT with Pr...,₱1.328 Million,2018,Diesel,Suv,NaN,First Owner,Automatic,Paranaque,7 seat,2199 CC,NaN
4,2018 Toyota Hiace 3.0 GL Grandia AT Detail,₱1.33 Million,2018,Diesel,Van,1 Km,First Owner,Automatic,Pasig,11 seat,2982 CC,NaN
5,2020 Toyota Fortuner 2.4 G Diesel 4x2 AT Detail,₱1.332 Million,2020,Diesel,Suv,"21,000 Km",First Owner,Automatic,Cebu City,7 seat,2393 CC,2020
6,2014 BMW 5 Series Sedan 520d Detail,₱1.338 Million,2014,Diesel,Sedan,"35,000 Km",First Owner,Automatic,Quezon City,5 seat,1995 CC,2014
7,2019 Nissan Terra 2.5 4x4 VL AT Detail,₱1.34 Million,2019,Diesel,Suv,"46,000 Km",First Owner,Automatic,Quezon City,7 seat,2488 CC,2019
8,2021 Toyota Hilux Conquest 2.4 4x2 A/T Detail,₱1.345 Million,2021,Diesel,Pickup-trucks,"22,000 Km",First Owner,Automatic,Quezon City,5 seat,2393 CC,2021
9,2017 Ford Explorer 2.3L Limited EcoBoost Detail,₱1.345 Million,2017,Gasoline,Suv,"26,000 Km",First Owner,Automatic,Quezon City,7 seat,2296 CC,2018


Under 1,500,000

In [ ]:
df_1_5m = df.copy()
df_1_5m = df_1_5m.drop_duplicates().reset_index(drop=True)
display(df_1_5m)
df_1_5m.to_csv("df_1_5m.csv")

,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,2016 Ford Explorer 3.5L 4x4 Limited+ Detail,₱1.42 Million,2016,7 seat,Gasoline,Suv,"40,000 Km",First Owner,Automatic,Las Piñas,3497 CC,NaN
1,2020 Toyota Hiace Commuter Deluxe Detail,₱1.42 Million,2020,15 seat,Diesel,Van,"18,000 Km",First Owner,Manual,Las Piñas,2755 CC,NaN
2,2019 Toyota Hilux Conquest 2.8 4x4 M/T Detail,₱1.428 Million,2019,5 seat,Diesel,Pickup-trucks,1 Km,First Owner,Manual,Quezon City,2755 CC,NaN
3,2020 Nissan Terra 2.5 4x2 VE AT Detail,₱1.428 Million,2020,7 seat,Diesel,Suv,"17,000 Km",First Owner,Automatic,Paranaque,2488 CC,2020
4,2019 Toyota Fortuner 2.4 V Diesel 4x2 AT Detail,₱1.428 Million,2019,7 seat,Diesel,Suv,"49,000 Km",First Owner,Automatic,Pasig,2393 CC,NaN
5,2014 Toyota FJ Cruiser 4.0L AT Detail,₱1.45 Million,2014,NaN,Gasoline,Suv,"19,000 Km",First Owner,Automatic,Paranaque,3998 CC,2014
6,2020 Jeep Renegade Longitude Detail,₱1.45 Million,2020,5 seat,Gasoline,Suv,"11,000 Km",First Owner,Automatic,Valenzuela,1398 CC,2020
7,2019 Ford Ranger Raptor 2.0L Bi-Turbo Detail,₱1.45 Million,2019,5 seat,Diesel,Pickup-trucks,NaN,First Owner,Automatic,Quezon City,1996 CC,NaN
8,2018 Mazda CX-9 2.5L Turbo Black Edition Detail,₱1.45 Million,2018,7 seat,Gasoline,Suv,"74,000 Km",First Owner,Automatic,Quezon City,2488 CC,2019
9,2018 Hyundai Grand Starex 2.5 Gold Dsl Detail,₱1.458 Million,2018,10 seat,Diesel,Van,1 Km,First Owner,Automatic,Quezon City,2497 CC,NaN


Under 2,000,000

In [ ]:
df_2m = df.copy()
df_2m = df_2m.drop_duplicates().reset_index(drop=True)
display(df_2m)
df_2m.to_csv("df_2m.csv")

,model,price,make_year,seats,fuel_type,body_type,ownership,transmission,city,engine_displacement,registration_year,kms_driven
0,2019 Nissan Terra 2.5 4x4 VL AT Detail,₱1.508 Million,2019,7 seat,Diesel,Suv,First Owner,Automatic,Paranaque,2488 CC,NaN,NaN
1,2023 Ford Ranger 2.0L Turbo Wildtrak 4x2 AT De...,₱1.518 Million,2023,5 seat,Diesel,Pickup-trucks,First Owner,Automatic,Quezon City,1996 CC,2023,"8,000 Km"
2,2016 Toyota 86 2.0 AT Detail,₱1.525 Million,2016,4 seat,Gasoline,Coupe,First Owner,Automatic,Marilao,1998 CC,NaN,"34,000 Km"
3,2018 Toyota Hiace 3.0 Super Grandia AT Detail,₱1.528 Million,2018,11 seat,Diesel,Van,First Owner,Automatic,Quezon City,2982 CC,NaN,1 Km
4,2015 Subaru WRX STI 2.5 MT Detail,₱1.55 Million,2015,5 seat,Gasoline,Sedan,Second Owner,Manual,Dumaguete city,2457 CC,2016,"20,000 Km"
...,...,...,...,...,...,...,...,...,...,...,...,...
102,2005 Hummer H2 6.0L Detail,₱2 Million,2005,NaN,Gasoline,Suv,Second Owner,Automatic,Pasig,NaN,2016,"45,000 Km"
103,2020 Mazda CX-9 2.5L Turbo AWD Exclusive Detail,₱2 Million,2020,7 seat,Gasoline,Suv,First Owner,Automatic,Quezon City,2488 CC,2020,"10,000 Km"
104,2016 Toyota Land Cruiser Prado 4.0L AT Detail,₱2 Million,2016,NaN,Gasoline,Suv,Second Owner,Automatic,Makati City,NaN,2018,"75,000 Km"
105,2016 Ford Explorer 3.5L Sport EcoBoost Detail,₱2 Million,2016,7 seat,Gasoline,Suv,Third Owner,Automatic,Baguio City,3496 CC,2020,"25,000 Km"


Under 3,000,000

In [ ]:
df_3m = df.copy()
df_3m = df_3m.drop_duplicates().reset_index(drop=True)
display(df_3m)
df_3m.to_csv("df_3m.csv")

,model,price,make_year,seats,fuel_type,body_type,ownership,transmission,city,engine_displacement,kms_driven,registration_year
0,2018 Toyota Hiace LXV Super Grandia 3.0 LXV A/...,₱2.038 Million,2018,10 seat,Diesel,Van,First Owner,Automatic,Pasig,2982 CC,NaN,NaN
1,2018 Toyota FJ Cruiser 4.0L V6 Detail,₱2.05 Million,2018,5 seat,Gasoline,Suv,First Owner,Automatic,Pasig,3956 CC,NaN,NaN
2,2018 Mercedes-Benz CLA-Class 200 Detail,₱2.09 Million,2018,5 seat,Gasoline,Sedan,First Owner,Dual-clutch,Las Piñas,1332 CC,"18,000 Km",NaN
3,2020 Mazda CX-5 2.0L FWD Pro Detail,₱2.1 Million,2020,5 seat,Gasoline,Crossover,First Owner,Automatic,Manila,1998 CC,"10,000 Km",2023
4,2019 Toyota Hiace GL Grandia A/T Detail,₱2.1 Million,2019,12 seat,Diesel,Van,First Owner,Automatic,Quezon City,2755 CC,"46,000 Km",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
69,2016 Land Rover Discovery Sport S 2.0L Diesel ...,₱2.95 Million,2016,7 seat,Diesel,Suv,Second Owner,Automatic,Pasig,1999 CC,"12,000 Km",2018
70,2018 Dodge Durango GT Detail,₱2.95 Million,2018,7 seat,Gasoline,Suv,First Owner,Automatic,Pasig,3598 CC,200 Km,2018
71,2023 Toyota Fortuner GR Sport 2.8L 4x4 AT Detail,₱2.95 Million,2023,7 seat,Diesel,Suv,First Owner,Automatic,Quezon City,2755 CC,NaN,NaN
72,2017 Mercedes-Benz E200 2.0 L AT Detail,₱2.998 Million,2017,NaN,Gasoline,Luxury-vehicles,First Owner,Automatic,Pasig,NaN,"23,000 Km",2017


Under 4000000

In [ ]:
df_4m = df.copy()
df_4m = df_4m.drop_duplicates().reset_index(drop=True)
display(df_4m)
df_4m.to_csv("df_4m.csv")

,model,price,registration_year,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement
0,2016 Chevrolet Suburban 4X2 LT Detail,₱3.05 Million,2016,2016,8 seat,Gasoline,Suv,"74,000 Km",First Owner,Automatic,Pasig,5328 CC
1,2017 Chevrolet Camaro 3.6L AT Detail,₱3.08 Million,2017,2017,NaN,Gasoline,Coupe,"11,000 Km",First Owner,Automatic,Quezon City,NaN
2,2020 Toyota Hiace LXV Super Grandia 3.0 LXV A/...,₱3.1 Million,2023,2020,10 seat,Diesel,Van,"20,000 Km",First Owner,Automatic,Quezon City,2982 CC
3,2019 Chevrolet Camaro 2.0L Turbo 3LT RS Detail,₱3.1 Million,2019,2019,4 seat,Gasoline,Coupe,"6,000 Km",Second Owner,Automatic,Pasig,1998 CC
4,2018 BMW 5 Series Gran Turismo 535i Detail,₱3.16 Million,2018,2018,5 seat,Gasoline,Sedan,"7,000 Km",First Owner,Automatic,Makati City,2979 CC
5,2019 BMW 5 Series Sedan 520d Detail,₱3.188 Million,NaN,2019,5 seat,Diesel,Sedan,"12,000 Km",First Owner,Automatic,Cavite City,1995 CC
6,2018 Toyota Land Cruiser Prado 4.0L Gas AT Detail,₱3.2 Million,2023,2018,7 seat,Gasoline,Suv,"38,000 Km",First Owner,Automatic,Manila,3956 CC
7,2016 Chevrolet Suburban 4X4 LTZ Detail,₱3.2 Million,2016,2016,7 seat,Gasoline,Suv,"35,000 Km",First Owner,Automatic,Pasig,5328 CC
8,2018 Mercedes-Benz E-Class Sedan E220 d Avantg...,₱3.23 Million,NaN,2018,5 seat,Diesel,Sedan,"40,000 Km",First Owner,Automatic,Las Piñas,1950 CC
9,2023 Hyundai Palisade 2.2 Diesel AT 4WD Detail,₱3.25 Million,2023,2023,7 seat,Diesel,Suv,"15,000 Km",Second Owner,Automatic,Manila,2199 CC


Under 5000000

In [ ]:
df_5m = df.copy()
df_5m = df_5m.drop_duplicates().reset_index(drop=True)
display(df_5m)
df_5m.to_csv("df_5m.csv")

,model,price,registration_year,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement
0,2022 Nissan Patrol Royale 5.6 V8 4x4 AT Detail,₱4.108 Million,2022,2022,8 seat,Gasoline,Suv,"13,800 Km",First Owner,Automatic,Pasig,5552 CC
1,2009 Porsche Cayman S Manual Detail,₱4.2 Million,NaN,2009,2 seat,Gasoline,Coupe,"32,000 Km",First Owner,Manual,Quezon City,3436 CC
2,2022 Audi e-tron Electric Detail,₱4.2 Million,2022,2022,5 seat,Electric,Suv,"30,000 Km",First Owner,Automatic,Paranaque,NaN
3,2019 Toyota Alphard 3.5 Gas AT Detail,₱4.25 Million,2019,2019,7 seat,Gasoline,Mpv,"50,000 Km",First Owner,Automatic,Silang,3456 CC
4,2017 Toyota Land Cruiser 200 4.5L Premium Detail,₱4.28 Million,2017,2017,8 seat,Diesel,Suv,"15,000 Km",First Owner,Automatic,Quezon City,4461 CC
5,2019 Mercedes-Benz C-Class Coupe C 300 Detail,₱4.35 Million,2021,2019,4 seat,Gasoline,Coupe,"30,000 Km",First Owner,Automatic,Las Piñas,1991 CC
6,2018 Toyota Land Cruiser 200 4.5L DSL AT Detail,₱4.36 Million,2018,2018,8 seat,Diesel,Suv,"40,000 Km",First Owner,Automatic,Quezon City,4461 CC
7,2015 Lexus LS L Detail,₱4.5 Million,2015,2015,NaN,Gasoline,Sedan,"60,000 Km",Second Owner,Automatic,Taguig,NaN
8,2020 Land Rover Range Rover Evoque SE 2.0 Dies...,₱4.55 Million,2023,2020,5 seat,Diesel,Suv,"7,000 Km",First Owner,Automatic,Apalit,1999 CC
9,2023 Hyundai Santa Fe 2.5T Calligraphy AWD Detail,₱4.6 Million,2023,2023,7 seat,Gasoline,Suv,"5,000 Km",First Owner,Dual-clutch,Manila,2497 CC


Over 5000000

In [ ]:
df_40m = df.copy()
df_40m = df_40m.drop_duplicates().reset_index(drop=True)
display(df_40m)
df_40m.to_csv("df_40m.csv")

,model,price,make_year,seats,fuel_type,body_type,kms_driven,ownership,transmission,city,engine_displacement,registration_year
0,2019 Chevrolet Camaro 6.2L AT Detail,₱5.28 Million,2019,4 seat,Gasoline,Coupe,900 Km,First Owner,Manual,Quezon City,NaN,NaN
1,2011 Shelby Mustang 5.0L AT Detail,₱5.28 Million,2011,NaN,Gasoline,Coupe,1 Km,First Owner,Automatic,Paranaque,4998 CC,NaN
2,2017 Shelby Mustang GT350 Detail,₱5.35 Million,2017,NaN,Gasoline,Coupe,1 Km,First Owner,Automatic,Mandaue City,NaN,2017
3,2021 BMW X5 xDrive30d Detail,₱5.35 Million,2021,5 seat,Diesel,Suv,"10,000 Km",First Owner,Automatic,Makati City,2993 CC,2022
4,2007 Chevrolet BT Cruiser 5.3L Detail,₱5.4 Million,2007,NaN,Gasoline,Suv,"6,000 Km",First Owner,Automatic,Pasig,NaN,2007
5,2014 Mercedes-Benz S-Class S500 4.7L Detail,₱5.4 Million,2014,NaN,Gasoline,Sedan,"21,000 Km",First Owner,Automatic,Quezon City,NaN,NaN
6,2018 Porsche 718 Boxster PDK Detail,₱5.5 Million,2018,2 seat,Gasoline,Convertibles,"12,000 Km",First Owner,Automatic,Makati City,1988 CC,2018
7,2018 Mercedes-Benz GL Class 350 BlueTec 4 Mati...,₱5.68 Million,2018,7 seat,Diesel,Suv,"4,000 Km",First Owner,Automatic,Cebu City,2987 CC,2018
8,2022 Land Rover Range Rover Velar S D200 With ...,₱5.8 Million,2022,5 seat,Diesel,Suv,"20,000 Km",First Owner,Automatic,Makati City,1999 CC,2022
9,2017 Cadillac Escalade ESV Platinum Detail,₱5.88 Million,2017,7 seat,Gasoline,Suv,NaN,First Owner,Automatic,Quezon City,5998 CC,NaN


Consolidated:

In [ ]:
all_data = pd.DataFrame()
all_data = pd.concat([df_100k, df_150k, df_200k, df_250k, df_300k, df_350k, df_400k, df_450k, df_500k, df_550k, df_600k, df_700k, df_650k, df_700k, df_750k, df_800k, df_850k, df_900k, df_950k, df_1m, df_1_1m, df_1_2m, df_1_3m, df_1_4m, df_1_5m, df_2m, df_3m, df_4m, df_5m, df_40m]).drop(['Unnamed: 0'], axis=1)
display(all_data)
all_data.to_csv("all_data.csv")

,model,price,registration_year,make_year,fuel_type,body_type,kms_driven,ownership,transmission,city,seats,engine_displacement
0,1990 Nissan Sentra 1.6L SGX MT Detail,"₱45,000",1990,1990,Gasoline,Sedan,"61,233 Km",First Owner,Manual,Quezon City,NaN,NaN
1,2021 Suzuki Celerio CVT 1.0L Detail,"₱60,000",2024,2021,Gasoline,Hatchback,"12,000 Km",First Owner,Cvt,Pasig,5 seat,998 CC
2,1994 Nissan Vanette 2.0L Vanette MT Detail,"₱70,000",2023,1994,Gasoline,Van,"84,710 Km",Second Owner,Manual,Quezon City,NaN,NaN
3,2000 Nissan Sentra 1.6L Super Saloon MT Detail,"₱70,000",2024,2000,Gasoline,Sedan,"150,000 Km",Second Owner,Manual,Mandaluyong City,NaN,NaN
4,1997 Mazda 3 Sedan 2.0L Premium Detail,"₱75,000",2019,1997,Gasoline,Sedan,"90,000 Km",Third Owner,Automatic,san-fernando,5 seat,1998 CC
...,...,...,...,...,...,...,...,...,...,...,...,...
31,2018 Rolls-Royce Ghost 6.6L Detail,₱18.5 Million,2018,2018,Gasoline,Sedan,"20,000 Km",First Owner,Automatic,Manila,NaN,NaN
32,2022 Cadillac Escalade ESV Platinum Detail,₱19 Million,2022,2022,Gasoline,Suv,10 Km,First Owner,Automatic,Manila,7 seat,5998 CC
33,2024 Ferrari F8 Tributo 3.9L Detail,₱20.5 Million,2024,2024,Gasoline,Coupe,"16,000 Km",First Owner,Automatic,Imus,2 seat,3902 CC
34,2023 Bentley Bentayga V8 Detail,₱21 Million,2023,2023,Gasoline,Suv,"30,000 Km",First Owner,Automatic,Olongapo City,5 seat,3996 CC
